In [75]:
import h5py

import re
import os

In [43]:
# Load the provided file
file_path = '/Users/fsd59835/globus/k11-45357.nxs' # for nix type OS


In [44]:
# Read and print the value of 'experiment_identifier'
with h5py.File(file_path, 'r') as file:
    start_time = file['entry/start_time'][()]
    beamline = file['entry/instrument/beamline'][()]
    experiment_no = file['entry/experiment_identifier'][()]
    nominal_energy = file['entry/instrument/configuration_summary/Imaging/nominal_energy'][()]
    config_summary_type = file['entry/instrument/configuration_summary/Imaging/type'][()]
    pco_z = file['entry/instrument/pco/pco_z'][()]
    flats = (file['entry/instrument/imaging/image_key'][()] == 1).sum()
    no_proj = (file['entry/instrument/imaging/image_key'][()] == 0).sum()
    darks = (file['entry/instrument/imaging/image_key'][()] == 2).sum()
    pco_exp = file['entry/instrument/imaging/count_time'][()]
    duration = (file['entry/duration'][()])/1000
   
    scan_request = file['entry/diamond_scan/scan_request'][()].decode('utf-8')

In [76]:
####some post processing

scan_no = os.path.splitext(os.path.basename(file_path))[0]

#convert seconds to Hr, mins
dur_hr = int(duration // 3600)
dur_min = int((duration % 3600) // 60)
dur_sec = duration % 60

# Convert the start time to a human-readable format
time_str = start_time.decode('utf-8')
start_timeHR= datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S.%f")

# Use regex to find the file path that follows the word "savu"
savu_path_match = re.search(r'savu"\s*:\s*"\s*(.*?)"', scan_request)
savu_path = savu_path_match.group(1) if savu_path_match else None

# Use regex to find all sequences of start, stop, and step values in the scan_request
theta_info = re.findall(r'start"\s*:\s*(-?\d+\.?\d*),\s*"stop"\s*:\s*(-?\d+\.?\d*),\s*"step"\s*:\s*(-?\d+\.?\d*)', scan_request)


In [88]:
print("Beamline : " + beamline.decode('utf-8'))  # Decoding from bytes to string
print(f"Experiment no. : {experiment_no.decode('utf-8')}\n") # Decoding from bytes to string

print("Scan no. : "+scan_no)
print("Start time : "+start_timeHR.strftime("%A, %B %d, %Y at %I:%M:%S %p"))
print(f"Duration : {dur_hr}:{dur_min}:{dur_sec:.1f}\n")

print("X-ray beam type : "+config_summary_type.decode('utf-8'))
print(f"Nominal energy (keV) : {nominal_energy}")
print (f"Propogation dist. (mm) : {pco_z}\n")

print(f"Exposure time (s) : {pco_exp}")
print(f"No. of flats : {flats}")
print(f"No. of darks : {darks}")
print(f"No. of projection : {no_proj}")
print(f"Start angle : {theta_info[0][0]}")
print(f"Stop angle : {theta_info[0][1]}")
print(f"Angular step : {theta_info[0][2]}\n")

print("savu PL : "+savu_path)

Beamline : k11
Experiment no. : mg36270-1

Scan no. : k11-45357
Start time : Wednesday, July 31, 2024 at 11:52:39 PM
Duration : 0:34:59.2

X-ray beam type : Monochromatic
Nominal energy (keV) : 14.07
Propogation dist. (mm) : 150.001

Exposure time (s) : 0.5
No. of flats : 40
No. of darks : 40
No. of projection : 4020
Start angle : 0.0
Stop angle : 180.0
Angular step : 0.04478726051256531

savu PL : /dls/k11/data/2024/mg36270-1/xml/tomography/configurations/Savu/full_fd_rr_vo_AST_tiff.nxs
